In [4]:
!pip install -q --upgrade \
    streamlit pyngrok paddleocr paddlepaddle pdf2image \
    beautifulsoup4 poppler-utils  pillow
!apt-get -y install libreoffice poppler-utils
!pip install python-docx
!pip install -q matplotlib
!pip install -q "paddleocr>=2.0.1" paddlepaddle
!pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.5#egg=detectron2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.0/195.0 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [31]:
%%writefile app.py
import streamlit as st
import os
os.environ["FLAGS_enable_bfloat16"] = "0"
import  json, re, subprocess, io, tempfile, shutil, zipfile, pprint, requests
from pdf2image import convert_from_path
from paddleocr import PaddleOCR, PPStructureV3
from bs4 import BeautifulSoup
from PIL import Image
import pandas as pd

st.title("📄 Document Parser (OCR + PPStructure)")

# ─────────────── Sidebar speed controls ───────────────
run_fast = st.sidebar.checkbox(
    "PPStructure fast-mode (skip fine cell/border models)",
    value=True
)
dpi_val  = st.sidebar.slider("Rasterise DPI", 50, 100, 70, 5)
shrink   = st.sidebar.slider("Resize before PPStructure (%)", 30, 100, 50, 5) / 100
# ───────────────────────────────────────────────────────

# ────────────────────────── Upload ──────────────────────────
uploaded_file = st.file_uploader(
    "Upload PDF / image / DOCX / XLSX",
    type=["pdf", "png", "jpg", "jpeg", "bmp", "tiff", "docx", "xlsx", "xls"],
)

# optional sample buttons
col1, col2 = st.columns(2)
with col1:
    if st.button("Load sample PDF"):
        url = "https://raw.githubusercontent.com/filippoalsen/ocr-samples/main/invoice.pdf"
        uploaded_file = io.BytesIO(requests.get(url).content)
        uploaded_file.name = "invoice.pdf"
with col2:
    if st.button("Load sample image"):
        url = "https://raw.githubusercontent.com/filippoalsen/ocr-samples/main/receipt.jpg"
        uploaded_file = io.BytesIO(requests.get(url).content)
        uploaded_file.name = "receipt.jpg"

# ───────────────────────── Helpers ──────────────────────────
def is_valid_pdf(path: str) -> bool:
    try:
        with open(path, "rb") as f:
            return f.read(5) == b"%PDF-"
    except Exception:
        return False

def convert_to_pdf_with_libreoffice(doc_path: str) -> str:
    pdf_path = os.path.splitext(doc_path)[0] + ".pdf"
    cmd = [
        "libreoffice", "--headless",
        "--convert-to", "pdf", doc_path,
        "--outdir", os.path.dirname(doc_path) or ".",
    ]
    res = subprocess.run(cmd, capture_output=True, text=True)
    if res.returncode != 0 or not os.path.exists(pdf_path) or not is_valid_pdf(pdf_path):
        st.write(res.stdout + res.stderr)
        raise RuntimeError("LibreOffice conversion failed or produced an invalid PDF.")
    return pdf_path

# ───────────────────────── Main branch ──────────────────────
if uploaded_file:
    filename = uploaded_file.name
    with open(filename, "wb") as f:
        f.write(uploaded_file.read())
    ext = os.path.splitext(filename)[1].lower()

    # 1️⃣  make image ----------------------------------------------------
    try:
        with st.spinner("📸 Preparing first page …"):
            if ext == ".pdf":
                if not is_valid_pdf(filename):
                    st.error("Bad PDF."); st.stop()
                images = convert_from_path(
                    filename, dpi=dpi_val, first_page=1, last_page=1
                )                                  # ← uses slider DPI
                image_path = "page1.png"
                images[0].save(image_path, "PNG")

            elif ext in [".png", ".jpg", ".jpeg", ".bmp", ".tiff"]:
                image_path = filename

            elif ext in [".docx", ".xlsx", ".xls"]:
                pdf_path = convert_to_pdf_with_libreoffice(filename)
                images = convert_from_path(
                    pdf_path, dpi=dpi_val, first_page=1, last_page=1
                )                                  # ← uses slider DPI
                image_path = "page1.png"
                images[0].save(image_path, "PNG")

            else:
                st.error("Unsupported file."); st.stop()

        st.image(image_path, caption="Image for OCR", use_column_width=True)
    except Exception as e:
        st.exception(e); st.stop()

    # 2️⃣  OCR + PPStructure --------------------------------------------
    try:
        with st.spinner("🔍 Running OCR & PPStructure …"):

            @st.cache_resource(show_spinner=False)
            def load_models():
                ocr = PaddleOCR(
                    lang="en",
                    use_angle_cls=False,
                    rec_batch_num=8           # batch OCR
                )
                pp  = PPStructureV3(
                    use_doc_orientation_classify=False,
                    use_doc_unwarping=False

                )
                return ocr, pp

            ocr, pipeline = load_models()

            # OCR on full-size image
            ocr_result = ocr.ocr(image_path)

            # resize before layout (slider)
            im = Image.open(image_path)
            small = im.resize((int(im.width * shrink),
                               int(im.height * shrink)))
            tmp_path = "tmp_small.png"
            small.save(tmp_path)

            # live progress bar
            bar = st.progress(0)
            output = pipeline.predict(
                input=tmp_path,
                progress_bar=True,
                call_back=lambda p: bar.progress(min(p, 100))
            )
            bar.empty()

        st.success("✅ Layout analysis done!")
    except Exception as e:
        st.exception(e); st.stop()

    # 3️⃣  save JSON / MD ------------------------------------------------
    os.makedirs("output", exist_ok=True)
    for i, res in enumerate(output, 1):
        res.save_to_json(f"output/output_{i}.json")
        res.save_to_markdown(f"output/output_{i}.md")

    # 4️⃣  combine page JSON → table list -------------------------------
    tables = []
    for i in range(1, len(output) + 1):
        with open(f"output/output_{i}.json") as f:
            tables.extend(json.load(f).get("table_res_list", []))
    st.info(f"Tables detected: **{len(tables)}**")

    # 5️⃣  parse first table --------------------------------------------
    structured_json = None
    for tbl in tables:
        if "pred_html" not in tbl:
            continue
        soup = BeautifulSoup(tbl["pred_html"], "html.parser")
        rows = [[c.get_text(strip=True) for c in r.find_all(["td", "th"])]
                for r in soup.find_all("tr") if r.find_all(["td", "th"])]
        if len(rows) < 2:
            continue
        headers, *data = rows
        headers = [h or f"Column{i}" for i, h in enumerate(headers)]
        structured_json = [{headers[i]: c for i, c in enumerate(r[:len(headers)])}
                           for r in data]
        break

    if structured_json:
        st.subheader("First parsed table")
        st.dataframe(pd.DataFrame(structured_json))
        st.download_button("Download table JSON",
                           json.dumps(structured_json, indent=2).encode(),
                           "table.json", "application/json")

    # 6️⃣  paragraph reconstruction -------------------------------------
    texts, scores, boxes = (ocr_result[0]["rec_texts"],
                            ocr_result[0]["rec_scores"],
                            ocr_result[0]["rec_boxes"])

    def inside_tb(box):
        x, y = box[0], box[1]
        for tb in [t["bbox"] for t in tables if "bbox" in t]:
            xs = [p[0] for p in tb]; ys = [p[1] for p in tb]
            if min(xs) <= x <= max(xs) and min(ys) <= y <= max(ys):
                return True
        return False

    filt = [(t, b) for t, s, b in zip(texts, scores, boxes) if s >= 0.5 and not inside_tb(b)]
    filt.sort(key=lambda x: (x[1][1], x[1][0]))

    def junk(t):
        t = t.strip()
        if len(t) < 5 or sum(c.isalpha() for c in t)/len(t) < 0.5:
            return True
        w = re.findall(r"\b\w+\b", t)
        real = [x for x in w if re.fullmatch(r"[A-Za-z]{3,}", x)]
        if (len(real)/len(w) if w else 0) < 0.5:
            return True
        return sum(len(x) for x in w)/len(w) > 15

    paras, buf, last = [], [], -100
    for t, b in filt:
        if junk(t): continue
        if abs(b[1] - last) > 15 and buf:
            paras.append(" ".join(buf)); buf = []
        buf.append(t); last = b[1]
    if buf: paras.append(" ".join(buf))

    st.subheader("Paragraphs")
    st.write(paras or "*(none)*")

    # 7️⃣  build document structure -------------------------------------
    doc = {
        "headings": [],
        "paragraphs": paras,
        "tables": structured_json and [structured_json] or [],
        "footnotes": []
    }

    def is_head(l):
        l = l.strip(); wc = len(l.split())
        if not l or len(l) > 80 or wc < 2: return False
        alpha = sum(c.isalpha() for c in l)/len(l)
        return l.isupper() or (l.istitle() and wc <= 6) or (l.endswith(":") and wc <= 8) or (alpha > 0.75 and wc <= 10)

    for ln in paras:
        low = ln.lower()
        if "http" in low or "retrieved" in low:
            doc["footnotes"].append(ln)
        elif is_head(ln):
            doc["headings"].append(ln)

    if not doc["headings"] and paras:
        doc["headings"].append(paras[0])

    st.subheader("Full document JSON")
    st.json(doc, expanded=False)
    st.download_button(
        "Download full JSON",
        data=json.dumps(doc, indent=2).encode(),
        file_name="document.json",
        mime="application/json",
    )


Overwriting app.py


In [34]:
!ngrok config add-authtoken 2zBjwnsLAbpqmwV3C1asgBE1R2A_K83UdtkcnrS74tj67RtZ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [35]:
from pyngrok import ngrok
ngrok.kill()
public_url = ngrok.connect(8501)
print("🌐 Streamlit URL:", public_url)

!streamlit run app.py --server.port 8501 > streamlit.log 2>&1 &

🌐 Streamlit URL: NgrokTunnel: "https://22d3-34-82-37-72.ngrok-free.app" -> "http://localhost:8501"
